In [1]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
import scipy.stats as sps

from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from scipy.interpolate import CubicSpline, UnivariateSpline, RectBivariateSpline
from vol import heston

warnings.filterwarnings('ignore')

In [9]:
r = 0.05
initial_price = 100

myheston = heston.Heston(s=initial_price, v=1, kappa=1, theta=0.4, sigma=0.4, rho=0.5, r=r)

T_grid = np.linspace(0.2, 1, 7)
K_grid = np.linspace(60, 140, 10)

observed_call_prices = myheston.call_price(T_grid.reshape(-1, 1), K_grid)
implied_volatilities = myheston.iv(T_grid.reshape(-1, 1), K_grid)

In [14]:
implied_volatilities.shape

(7, 10)

In [5]:
def get_trajectory(S, T_extended):
    """
    Args:
        S: extended price grid (== strike grid), vector
    """
    iterations = 0
    MAX_ITERATIONS = 100
    
    while iterations < MAX_ITERATIONS:
        stock_prices = myheston.simulate_euler(T_extended.max(), len(T_extended), 1)
        if stock_prices.min() >= S[0] and stock_prices.max() <= S[-1]:
            return stock_prices.flatten()
        iterations += 1
        
    print('cannot simulate')


In [6]:
def delta_function(stock_price, K, S, V, iv, t_max, t, method, r):
    """
    IMPORTANT: V is a vector here
    """
    if method != 'BS':
        spline = CubicSpline(S, V, bc_type='natural')
        return spline.derivative(nu=1)(stock_price)
    
    spline_theta = CubicSpline(S, iv, bc_type='natural')
    volatility = spline_theta(stock_price)
    d1_num = np.log(stock_price / K) + (volatility * volatility / 2 + r) * (t_max - t)
    d1_denum = volatility * np.sqrt(t_max - t)
    return sps.norm.cdf(d1_num / d1_denum)
    
    
def call_price_function(stock_price, S, V):
    spline = CubicSpline(S, V, bc_type='natural')
    return spline(stock_price)

def hedge(K, S, V, iv, stock_prices, call_price_function, delta_function, times, method, r = 0.05):
    """
    Args:
        K: strike, scalar
        S: extended strike grid, vector
        V: call prices, matrix
        iv: implied volatility on the extended grid, matrix
        stock_prices: trajectory, vector indexed by time
        call_price_function: pricing function for a call option
        delta_function: function to obtain delta
        times: extended time grid, vector
        method: 'BS' -- Black-Scholes, local volatility otherwise
        r: interest rate
    Returns:
        pnl: pnl trajectory, vector
        call prices: theoretical call price trajectory, vector
    """
    money_account = call_price_function(stock_prices[0], S, V[0])
    delta = 0
    pnl = []
    V_trajectory = []
    
    for i in range(len(V) - 1):
        new_delta = delta_function(stock_prices[i], K, S, V[i], iv[i], times[-1], times[i], method, r)
        money_account -= (new_delta - delta) * stock_prices[i]
        money_account *= np.exp(r * (times[i + 1] - times[i]))
        delta = new_delta
        
        pnl.append(money_account + delta * stock_prices[i])
        V_trajectory.append(call_price_function(stock_prices[i], S, V[i]))
        
    pnl, V_trajectory = np.array(pnl), np.array(V_trajectory)
    
    return pnl, V_trajectory

In [11]:
realized_stock_prices = get_trajectory(K_grid, T_grid)
realized_stock_prices

array([100.        ,  93.55607238,  81.23718611,  77.58512656,
       118.47543724,  99.7580598 ,  80.71961426,  79.84585939])

In [12]:
for i in range(T_grid):
    t=T_grid[i]
    Stek=realized_stock_prices[i]

0.2
0.33333333333333337
0.4666666666666667
0.6000000000000001
0.7333333333333334
0.8666666666666667
1.0


In [ ]:

pnl, V_trajectory = hedge(
    120, K[0], V, lv.get_extended_iv(),
    stock_prices, call_price_function, delta_function,
    T_extended[:, 0], 'BS', r
)